In [4]:
import numpy as np

# Data Preparation

In [5]:
import data_module as dm
DLS_X, DLS_Y, DLS_W, DLD_X, DLD_Y, DLD_W = dm.import_data()
#DLS = data large scale, contains thousands of datapoints
#DLD = data low dimension, contains less than or hundreds of datapoints

In [6]:
#select DLD first, as we gonna try to build up the model (faster execution)
data_w = DLD_W[0] #maximum weight
data_x = DLD_X[0] #datapoints
data_y = DLD_Y[0] #optimum values

# Solution Representation
The solution is represented as boolean mask of $N$ size, where $N$ is the size of data. 1 or True means the consecutive data is selected / carried away. 0 or False menas the consecutive data is not selected/carried away

In [7]:
sol = np.random.random(len(data_x))
sol

array([0.26883136, 0.58847016, 0.33888147, 0.60731337, 0.86338164,
       0.93504649, 0.74749793, 0.72260778, 0.7234924 , 0.28433166,
       0.5799968 , 0.96382489, 0.52424693, 0.27918827, 0.25235398,
       0.09644272, 0.00990284, 0.4418656 , 0.31693068, 0.5861497 ])

In [8]:
#after discretizatoin
np.round(sol)

array([0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 1.])

In [9]:
idx = np.arange(len(data_x))
idx = idx[np.round(sol).astype(bool)]
carried = data_x.iloc[idx]
carried

,weight,price
2,72,83
4,46,4
5,55,44
6,8,6
7,35,82
8,75,92
9,61,25
11,77,56
12,40,18
13,63,58


# Fitness Function

In [10]:
def fit(sol, data, max_w) :
    #Discretization
    mask = np.round(sol).astype(bool)
    
    #decode
    idx = np.arange(len(data))
    idx = idx[mask]
    data_solution = data.iloc[idx]
    
    #price 
    price = np.sum(data_solution['price'])
    weight = np.sum(data_solution['weight'])
    if weight <= max_w:
        return price
    else :
        return 0

In [11]:
# Hyper parameters 
fmin = 0
fmax = 1
A = 1
alpha = 0.98
gamma = 0.98
population = 20

In [12]:
# def init_x (population, data_size):
#     return np.random.random((population,data_size))

# def init_f (population, fmin,fmax):
#     return  np.random.uniform(fmin,fmax,population)

# def init_v (poulation):
#     return np.zeros(population)

# def init_y (solutions,data):
#     #solutions : list of solution
#     Y = np.zeros(len(solutions))
#     for i,sol in enumerate(solutions) : 
#         Y[i] = fit(sol,data)
#     return Y

# def init_r (population):
#     return np.random.random(population)

# def init_A (population):
#     # Aku lupa A itu apa
#     return None

In [75]:
class bat:
    def __init__(self, population, data, max_w, fmin, fmax, A, alpha, gamma):
        self.population = population
        self.data = data
        self.max_w = max_w
        self.fmin = fmin
        self.fmax = fmax
        self.A = A
        self.alpha = alpha
        self.gamma = gamma
        self.data_size = len(data)
        self.best_sol = None
        
        self.init_x()
        self.init_f()
        self.init_v()
        self.init_y()
        self.init_r()
                
    def init_x (self):
        self.solutions =  np.random.random((self.population,self.data_size))
        
    def init_f (self):
        self.f = np.random.uniform(self.fmin,self.fmax,self.population)

    def init_v (self):
        self.v = np.zeros((self.population, self.data_size))

    def init_y (self):
        Y = np.zeros(len(self.solutions))
        for i,sol in enumerate(self.solutions) : 
            Y[i] = fit(sol,self.data, self.max_w)
        
        self.Y = Y

    def init_r (self):
        self.r = np.random.random(self.population)
        
    
    def update_f(self):
        self.fmin = np.min(self.f)
        self.fmax = np.max(self.f)
        betha = np.random.random(len(self.f))
        self.f = betha*(self.fmax-self.fmin) + self.fmin
    
    def update_v(self):
        self.find_best_solution()
        r = (self.solutions - self.best_sol)
        rr = [r[i] * baat.f[i] for i in range(len(r))] # perkalian r dengan f
        
        self.v = self.v + rr
    
    def update_x(self):
        self.solutions += self.v
    
    #__________________
    def find_best_solution(self):
        self.best_sol = self.solutions[np.argmax(self.Y)]

In [82]:
baat = bat(10,data_x,data_w,fmin,fmax,A,alpha,gamma)

In [83]:
# def train(bat, epoch):
#     for i in range(epoch):
#         #update kelelawar
        

# ini line coba

In [85]:
baat.solutions.shape

(10, 20)

In [87]:
baat.f

array([0.40849602, 0.4550697 , 0.63402954, 0.51261623, 0.84757873,
       0.83249064, 0.98522798, 0.63150433, 0.55785305, 0.24440975])

In [88]:
baat.update_f()

In [89]:
baat.f

array([0.45095719, 0.77874818, 0.28309018, 0.35866644, 0.32210485,
       0.24877135, 0.72690442, 0.43384982, 0.73107937, 0.44363727])

In [91]:
baat.v.shape

(10, 20)

In [94]:
baat.update_v()
baat.v

array([[-0.17222884, -0.2898008 , -0.07717717, -0.17424419, -0.47198584,
        -0.20806053,  1.15451558,  0.46703157, -0.57073202, -0.18200111,
         0.30082647,  0.26351575,  0.17755438, -0.73708875, -0.29160625,
        -0.7153411 , -0.49155094, -0.96478454, -0.79877989, -0.90187088],
       [-0.38353062, -0.84902131,  1.14710007, -0.75672833, -0.69258168,
        -0.87820194,  0.69854086,  1.67521123, -0.58005214, -0.94861524,
         0.97447339, -0.40801241, -1.54411298,  0.80400331, -0.29029957,
        -1.6400605 , -1.14579938, -1.50105068, -0.00633694, -1.38297755],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.24782944, -0.31082071, -0.1137984 , -0.35370549, -0.06274649,
        -0.2829052 ,  0.15209052,  0.80338344,  

In [100]:
baat.solutions.shape

(10, 20)

In [99]:
baat.update_x()
baat.solutions.shape

(10, 20)